In [1]:
from ultralytics import YOLO
import os
from IPython.display import Image, display
from IPython import display
import torch
from collections import defaultdict
import numpy as np
import cv2

In [ ]:
model_path = 'model_checkpoint/weights/best.pt'
model = YOLO(model_path) #This is to load previously trained weights
 # model = YOLO('yolov11n.pt') #Use this to load the default YOLO


In [3]:
classes = ['Car', 'Cyclist', 'Pedestrian']  # Replace with your actual class names
train_path = r'YOLO\images\train'  # Updated training dataset path
valid_path = r'YOLO\images\val'  # Replace with your actual validation dataset path

yaml_file = 'names:\n'
yaml_file += '\n'.join(f'- {c}' for c in classes)
yaml_file += f'\nnc: {len(classes)}'
yaml_file += f'\ntrain: {str(train_path)}\nval: {str(valid_path)}'
with open('kitti.yaml', 'w') as f:
    f.write(yaml_file)

In [ ]:
train_results = model.train(
    data='kitti.yaml', # Path to the dataset configuration file,
    epochs=300, # Increased epochs
    patience=0, # No early stopping
    batch=16,# Batch size
    imgsz=640,  # Image size
    mixup=0.1,# Mixup augmentation factor
    project='Perception-yolov11n-kitti', # Project name
    name='Training',# Naming the experiment
    resume = False,# Resume training from the last checkpoint
    device=0)# Use GPU if available, specified as a string

In [ ]:
sequences = [1,2,3]
yolo_classes = ['Car', 'Pedestrian', 'Cyclist']
image_width = 1242
image_height = 370

base_dir = r"D:\Chinmay\MSc - DTU\Course Material\3rd Sem\Perception\Project\34759_final_project_rect"
for seq_number in sequences:
    model = YOLO(model_path)
    seq_dir = os.path.join(base_dir, f"seq_{seq_number:02d}", "image_02", "data")

    image_list = sorted([os.path.join(seq_dir, f) for f in os.listdir(seq_dir) if f.endswith(('.png', '.jpg'))])

    output_file = f"tracking_results_seq_{seq_number:02d}.txt"
    track_history = defaultdict(lambda: [])

    # for image in image_list:
    #     results = model.track(image, persist=True)
    #     boxes = results[0].boxes.xywh.cpu()
    #     track_ids = results[0].boxes.id.int().cpu().tolist()

    with open(output_file, "w") as f:
        for image_path in image_list:
            # Get the filename from the path
            image_name = os.path.basename(image_path)
            
            # Perform tracking on the image
            results = model.track(image_path, persist=True,tracker="bytetrack.yaml")
            
            # Process and save the results
            for result in results:
                boxes = result.boxes.xywh.cpu().numpy()  # Bounding boxes in [x, y, w, h] format
                class_ids = result.boxes.cls.cpu().numpy()  # Class IDs
                scores = result.boxes.conf.cpu().numpy()  # Confidence scores
                track_ids = result.boxes.id.cpu().numpy() if result.boxes.id is not None else [-1] * len(boxes) # Track IDs
                
                # Write results for each object
                for box, cls_id, track_id, score in zip(boxes, class_ids, track_ids,scores):
                    cls_id = int(float(cls_id))
                    track_id = int(track_id)
                    x, y, w, h = box[0], box[1], box[2], box[3]
                    x, y, w, h = map(float, [x, y, w, h])
                    x1, y1 = int(x - w / 2), int(y - h / 2)
                    x2, y2 = int(x + w / 2), int(y + h / 2)

                    # PLACEHOLDERS
                    truncated = 0  # Placeholder
                    occluded = 0   # Placeholder
                    alpha = -10.0  # Placeholder
                    bbox = f"{x1:.2f} {y1:.2f} {x2:.2f} {y2:.2f}"
                    dimensions = "0.0 0.0 0.0"  # Placeholder for height, width, length
                    location = "0.0 0.0 0.0"    # Placeholder for 3D coordinates
                    rotation_y = "-10.0"        # Placeholder for rotation

                    class_name = yolo_classes[cls_id]
                    score_value = score if isinstance(score, float) else float(score)

                    f.write(f"{image_name.split('.')[0]} {track_id} {class_name} {truncated} {occluded} {alpha} {x1:.2f} {y1:.2f} {x2:.2f} {y2:.2f} {dimensions} {location} {rotation_y} {score_value:.3f}\n")